In [ ]:
import sys

def shadow(limit=200):
    def decorator(generator_func):
        def wrapper_gen(*args, **kwargs):
            total_sum = 0
            limit_triggered = False
            
            original_gen = generator_func(*args, **kwargs)
            
            while True:
                try:
                    item = next(original_gen)
                    
                    yield item
                    parts = item.split()
                    
                    if (len(parts) == 2 and 
                        parts[0] in ("payment", "refund", "transfer") and 
                        parts[1].isdigit()):
                        
                        amount = int(parts[1])
                        total_sum += amount
                        
                        if total_sum > limit and not limit_triggered:
                            print(
                                f"\n--- (Тіньовий ліміт {limit} перевищено. Активую схему) ---\n", 
                                file=sys.stderr
                            )
                            limit_triggered = True

                except StopIteration:
                    return total_sum
        
        return wrapper_gen
    return decorator

@shadow(limit=200)
def get_transactions():
    print("--- Початок потоку транзакцій ---")
    yield "payment 120"
    yield "refund 50"
    yield "invalid line to be ignored"
    yield "transfer 80"  
    yield "payment 20"
    yield "not a number 10x"
    yield "payment 100"  
    print("--- Кінець потоку транзакцій ---")

final_total = 0
print("Активую обгорнутий генератор...\n")

try:
    transaction_stream = get_transactions()
    
    while True:
        item = next(transaction_stream)
        print(f"Оброблено: {item}")
        
except StopIteration as e:
    final_total = e.value
    print("\n...Генератор вичерпано.")

print("=" * 30)
print(f"Фінальна тіньова сума: {final_total}")
print("=" * 30)

Активую обгорнутий генератор...

--- Початок потоку транзакцій ---
Оброблено: payment 120
Оброблено: refund 50
Оброблено: invalid line to be ignored
Оброблено: transfer 80
Оброблено: payment 20
Оброблено: not a number 10x
Оброблено: payment 100
--- Кінець потоку транзакцій ---

...Генератор вичерпано.
Фінальна тіньова сума: 370



--- (Тіньовий ліміт 200 перевищено. Активую схему) ---

